In [1]:
# https://www.youtube.com/watch?v=XkjpPecU2e8&ab_channel=FinancialProgrammingwithRitvik


In [2]:
import os, sys
from google.colab import drive
drive.mount('/content/mnt')
nb_path = '/content/notebooks'
os.symlink('/content/mnt/My Drive/Colab Notebooks', nb_path)
sys.path.insert(0, nb_path)  # or append(nb_path)

Mounted at /content/mnt


In [3]:
# Install only once. Tomorrow, you can skip this.
!pip install websocket-client

     |████████████████████████████████| 71kB 9.1MB/s 


In [4]:
import websocket
import numpy as np
import requests
import json


In [5]:
# !pip install jupyterthemes
# !jt -t gruvboxd -N -T

In [6]:
symbol_1 = 'ADAUSDT'
symbol_2 = 'adausdt'
interval = '1m'

root_url = 'https://api.binance.com/api/v1/klines'
url = root_url + '?symbol=' + symbol_1 + '&interval=' + interval
data = json.loads(requests.get(url).text)


In [7]:
data = data[-30:]

In [8]:
data

[[1621745760000,
  '1.47710000',
  '1.47830000',
  '1.47440000',
  '1.47450000',
  '300602.84000000',
  1621745819999,
  '443664.14252200',
  762,
  '79939.12000000',
  '117959.25266600',
  '0'],
 [1621745820000,
  '1.47450000',
  '1.47610000',
  '1.47200000',
  '1.47250000',
  '261358.38000000',
  1621745879999,
  '385135.12736600',
  882,
  '107468.62000000',
  '158347.88619500',
  '0'],
 [1621745880000,
  '1.47250000',
  '1.47480000',
  '1.47200000',
  '1.47250000',
  '199263.93000000',
  1621745939999,
  '293534.64207300',
  729,
  '96923.25000000',
  '142779.28262100',
  '0'],
 [1621745940000,
  '1.47270000',
  '1.47650000',
  '1.47270000',
  '1.47470000',
  '265761.22000000',
  1621745999999,
  '391849.93289400',
  681,
  '168998.26000000',
  '249143.66841000',
  '0'],
 [1621746000000,
  '1.47500000',
  '1.47510000',
  '1.46370000',
  '1.46420000',
  '839832.94000000',
  1621746059999,
  '1234185.93577300',
  1399,
  '181467.66000000',
  '266456.80173800',
  '0'],
 [1621746060000

In [9]:
socket = f'wss://stream.binance.com:9443/ws/{symbol_2}@kline_{interval}'

In [10]:
amount_left = 10
investment  = 10
avg_26, avg_9 = 0, 0
amt_coin = 0
portfolio = 0


local_lowest = 1000000000000
local_highest = 0
global_lowest = 1000000000000
global_highest = 0

opens, highs, lows, closes  = [], [], [], []

In [11]:
for i in data:
  opens.append(i[1])
  highs.append(i[2])
  lows.append(i[3])
  closes.append(i[4])

In [12]:
opens = [float(i) for i in opens] 
highs = [float(i) for i in highs] 
lows = [float(i) for i in lows] 
closes = [float(i) for i in closes] 

In [13]:
# Buying and Selling functions

def buy(allocated_money, price):
  global portfolio, amount_left
  amt_coin = allocated_money/price

def sell(allocated_money, price):
  global portfolio, amount_left
  amt_coin = 0


In [14]:
def on_message(ws,message):
    global  investment, portfolio, amount_left, amt_coin, opens, closes, highs, lows 
    global local_highest, local_lowest
    global avg_26, avg_9
    json_message = json.loads(message)
    cs = json_message['k']
    candle_closed, close, high, low = cs['x'], cs['c'], cs['h'], cs['l']
    print(candle_closed)

    if candle_closed:
        closes.append(float(close))
        highs.append(float(high))
        lows.append(float(low))
        last_price = closes[-1]
        print(f'Closes: {closes}')
        
        if(last_price < local_lowest):
            local_lowest = last_price
        if(last_price > local_highest):
            local_highest = last_price
        print(f'Lowest_Point: {local_lowest}')
        print(f'Highest_Point: {local_highest}')
        
        if(last_price < closes[-4]):
            local_lowest = last_price
        if(last_price > closes[-4]):
            local_highest = last_price

        portfolio = amt_coin*last_price
        avg_26 = sum(closes[-26:])/26
        avg_9 = sum(closes[-9:])/9
        
        
        if ((amount_left != 0) and (avg_9 > avg_26)):
            buy(amount_left, price=last_price)
            print(f'We bought ${amount_left} worth of bitcoin', '\n', '\n')
            portfolio =  amount_left
            amt_coin = amount_left/last_price
            amount_left = 0
            

        elif ((amount_left == 0) and (avg_9 < avg_26)):
            sell(-portfolio, price=last_price)
            print(f'We sold ${-portfolio} worth of bitcoin', '\n', '\n') 
            amount_left = portfolio
            portfolio = 0
            amt_coin = 0
            
          
        print(f'Avg 9: ${avg_9}')  
        print(f'Avg 26: ${avg_26}')  
        print(f'Current amount left ${amount_left}')
        print(f'Current portfolio worth ${portfolio}')
        print(f'Current quantity of coins ${amt_coin}')
        print((local_highest-local_lowest)/last_price*100)
        print("\n")
    
    
ws = websocket.WebSocketApp(socket, on_message=on_message)

In [ ]:
ws.run_forever()

False
False
False
False
False
False
False
False
False
False
True
Closes: [1.4745, 1.4725, 1.4725, 1.4747, 1.4642, 1.4606, 1.4574, 1.4574, 1.4557, 1.4511, 1.4488, 1.4474, 1.4466, 1.45, 1.4464, 1.4459, 1.4463, 1.4484, 1.4493, 1.4458, 1.4419, 1.4371, 1.4423, 1.436, 1.4443, 1.4427, 1.4401, 1.4511, 1.4546, 1.4525, 1.452]
Lowest_Point: 1.452
Highest_Point: 1.452
Avg 9: $1.4461777777777778
Avg 26: $1.4481423076923075
Current amount left $10
Current portfolio worth $0.0
Current quantity of coins $0
0.0


False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
Closes: [1.4745, 1.4725, 1.4725, 1.4747, 1.4642, 1.4606, 1.4574, 1.4574, 1.4557, 1.4511, 1.4488, 1.4474, 1.4466, 1.45, 1.4464, 1.4459, 1.4463, 1.4484, 1.4493, 1.4458, 1.4419, 1.4371, 1.4423, 1.436, 1.4443, 1.4427, 1.4401, 1.4511, 1.4546, 1.4525, 1.452, 1.4467]
Lowest_Point: 1.4467
Highest_Point: 1.452
Avg 9: $1.4466666666666668
A